# Get track uris

## Import all the things

In [1]:
import pandas as pd

import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials # will update so user can create their own playlist

## Get Spotify API credentials

Create a text file named creds.txt in the same folder as this notebook. On the top line put your client id and on the second put your secret id. Don't add a return at the end of the secret id.

In [2]:
# read Spotify credentials from txt file
with open("creds.txt") as f:
    creds = f.readlines()

In [3]:
cid = creds[0][:-1]   # 0 means top number and -1 gets rid of \n
secret = creds[1]     # second number

In [4]:
# Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = sp.Spotify(client_credentials_manager = client_credentials_manager)

## Function that does all the work

In [5]:
def get_songs(genre):
    '''Function to get 1000 songs of any genre from spotify
       input: genre wanted (see readme to get a link to a list of genres)
       output: dataframe that includes artist, track name, and track uri
    '''
    
    # create empty lists to append track information to
    artist_name = []
    track_name = []
    track_id = []
    
    # for loop for offset value
    # offset value makes it get songs from multiple pages
    for i in range(0, 1000, 50):
        track_results = sp.search(q="genre:" + genre, limit = 50, type='track', offset=i) 
        
        # get information for all tracks and append to lists above. 
        for i, t in enumerate(track_results['tracks']['items']):
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
        
    # dictionary created with lists saved to dictionary. 
    songs = {}
    songs['artist'] = artist_name
    songs['track_name'] = track_name
    songs['track_id'] = track_id
    songs['genre'] = genre
    
    # dictionary saved as data frame.
    songs = pd.DataFrame(songs)
    
    return songs

## Call function and clean data

In [6]:
# call function to get songs
songs_df = get_songs('glam metal')

In [7]:
# drop all live and demo version
songs_df = songs_df[~songs_df['track_name'].str.contains("Live")]
songs_df = songs_df[~songs_df['track_name'].str.contains("live")]
songs_df = songs_df[~songs_df['track_name'].str.contains("Demo")]
songs_df = songs_df[~songs_df['track_name'].str.contains("demo")]

In [8]:
# sort by artist
songs_df = songs_df.sort_values('artist')
songs_df.head()

,artist,track_name,track_id,genre
69,Accept,Too High to Get It Right,0rDhrV26w2t2pwB68sE8J7,glam metal
144,Accept,Slaves To Metal,5PKCaK2R5hNyApWqXiZHYo,glam metal
58,Accept,Screaming for a Love-Bite,67di8AD3Pkshu40iQOjKzD,glam metal
74,Accept,Up to the Limit,46VXd3fLT4oQdB2CuBRVzG,glam metal
76,Accept,Living for Tonite,2fsdgJ8HPXkH2btaqijj3x,glam metal


In [9]:
# get a list of artists
artists = songs_df.artist.unique()
artists

array(['Accept', 'Ace Frehley', 'Airbourne', 'Alice Cooper',
       'Baton Rouge', 'Beautiful Creatures', 'Bon Jovi', 'Britny Fox',
       'Buckcherry', 'Bulletboys', 'Cheap Trick', 'Cinderella',
       'Crash Kelly', "D'Molls", 'Damn Yankees', 'Dangerous Toys',
       'Dee Snider', 'Diamond Rexx', 'Dio', 'Dirty Looks', 'Dokken',
       'Electric Angels', 'Electric Boys', "Enuff Z'Nuff", 'Eric Carr',
       'Europe', 'Extreme', 'Exxocet', 'Faster Pussycat', 'Firehouse',
       'George Lynch', 'Great White', 'Heavy Metal Kids', 'Helix',
       'Hericane Alice', 'House Of Lords', 'Iggy Pop', 'Jani Lane',
       'Jeff Scott Soto', 'Joan Jett & The Blackhearts', 'KISS', 'Keel',
       'King Kobra', 'Kingdom Come', 'Kip Winger', "Kissin' Dynamite",
       'Kix', 'Krokus', 'L.A. Guns', 'LOUDNESS', 'Lee Aaron', 'Lita Ford',
       'Lizzy Borden', 'Lou Reed', 'Lynch Mob', 'Madam X',
       'Mark Slaughter', 'Metal Shop', 'Michael Monroe', 'Mother Feather',
       'Nazareth', 'Nelson', 'New Yor

In [13]:
# count songs by each artist
num_songs = songs_df.value_counts('artist')
num_songs

artist
Heavy Metal Kids               116
Faster Pussycat                 39
Saxon                           38
Quiet Riot                      35
Bulletboys                      30
                              ... 
Robin McAuley                    1
Kip Winger                       1
Joan Jett & The Blackhearts      1
Keel                             1
Zebra                            1
Length: 116, dtype: int64

In [11]:
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 912 entries, 69 to 7
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artist      912 non-null    object
 1   track_name  912 non-null    object
 2   track_id    912 non-null    object
 3   genre       912 non-null    object
dtypes: object(4)
memory usage: 35.6+ KB
